In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-05-23 13:39:17--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  1.22MB/s    in 0.8s    

2022-05-23 13:39:19 (1.22 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|                Y|Great organizer, ...|Very nice. Will o...| 2015-08-31|
|         US|   47910848|  RF0D1LEIF6L7|B001TJGCS0|     570955425|Nite Ize Nite Daw...|           Tools|          4|    

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
clean_df = df.dropna()

In [6]:
vine_df = clean_df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2UM5QMHBHC90Q|          5|            0|          0|   N|                Y|
|  RF0D1LEIF6L7|          4|            0|          0|   N|                Y|
|  RM6YKIWQVNSY|          1|            6|          6|   N|                Y|
|R1RL3L68ASPS36|          4|            0|          0|   N|                Y|
|R1U4XFBFAG34CY|          5|            0|          0|   N|                Y|
|R3KFIK8P0I91PL|          5|            0|          0|   N|                Y|
| RENOAY76PPK1O|          5|            0|          0|   N|                Y|
| RINV884I0NL5V|          1|            0|          0|   N|                Y|
| R5KJH6CXZH2PX|          5|            0|          0|   N|                Y|
| RO69JF6QWD0W1|          5|            0|          0|   N|     

In [7]:
vote_count_20 = vine_df.filter("total_votes >= 20")
vote_count_20.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RXAHWIC1584UQ|          5|           20|         23|   N|                Y|
| RSBELAIC899DO|          5|           46|         51|   N|                Y|
|R2HCC4CJ59D225|          4|           43|         43|   N|                Y|
|R39BVCCVPRV6F5|          5|           10|         22|   N|                Y|
| RL9VF9WXHEHKR|          5|           35|         36|   N|                Y|
|R1ZINLWUOLTZ46|          1|           39|         45|   N|                N|
|R1CQKM1K1CHOHS|          1|           21|         22|   N|                N|
|R1ZG19X1NOLCRF|          5|            5|         21|   N|                Y|
|R1YOX5Z4GF3ZSW|          1|            1|         22|   N|                Y|
| R2DHAM7J1KMWN|          5|           23|         24|   N|     

In [8]:
vote_count_50 = vote_count_20.filter("(helpful_votes/total_votes)>=0.5")
vote_count_50.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RXAHWIC1584UQ|          5|           20|         23|   N|                Y|
| RSBELAIC899DO|          5|           46|         51|   N|                Y|
|R2HCC4CJ59D225|          4|           43|         43|   N|                Y|
| RL9VF9WXHEHKR|          5|           35|         36|   N|                Y|
|R1ZINLWUOLTZ46|          1|           39|         45|   N|                N|
|R1CQKM1K1CHOHS|          1|           21|         22|   N|                N|
| R2DHAM7J1KMWN|          5|           23|         24|   N|                N|
|R26EZ2INBETPU2|          1|          103|        128|   N|                N|
|R265ZMDO2ISN1O|          5|           20|         21|   N|                Y|
|R3NZ7JTOCPYE1S|          4|           32|         33|   N|     

In [9]:
paid_reviews = vote_count_50.filter("vine == 'Y'")
paid_reviews.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3QIQRG107AP19|          5|           38|         41|   Y|                N|
|R1ZNBHGCYE8VN1|          5|          175|        181|   Y|                N|
|R1BNZJ9IKGRBC2|          4|           63|         70|   Y|                N|
|R2IXTN4PBRHBO2|          5|           19|         24|   Y|                N|
|R32AOW3L09SYMQ|          5|           29|         34|   Y|                N|
|R34Z4VTLF7PC6X|          4|           33|         36|   Y|                N|
|R25403NE9JCRZZ|          2|           36|         40|   Y|                N|
|R2QT2G3YRGC0PS|          5|           18|         20|   Y|                N|
| RDUP15OURNUSA|          5|           22|         23|   Y|                N|
|R1EI4NZTG7G14F|          5|           17|         23|   Y|     

In [10]:
unpaid_reviews = vote_count_50.filter("vine == 'N'")
unpaid_reviews.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RXAHWIC1584UQ|          5|           20|         23|   N|                Y|
| RSBELAIC899DO|          5|           46|         51|   N|                Y|
|R2HCC4CJ59D225|          4|           43|         43|   N|                Y|
| RL9VF9WXHEHKR|          5|           35|         36|   N|                Y|
|R1ZINLWUOLTZ46|          1|           39|         45|   N|                N|
|R1CQKM1K1CHOHS|          1|           21|         22|   N|                N|
| R2DHAM7J1KMWN|          5|           23|         24|   N|                N|
|R26EZ2INBETPU2|          1|          103|        128|   N|                N|
|R265ZMDO2ISN1O|          5|           20|         21|   N|                Y|
|R3NZ7JTOCPYE1S|          4|           32|         33|   N|     

In [11]:
total_reviews = vote_count_50.count()
total_reviews

31827

In [12]:
paid_reviews_T = vote_count_50.filter("vine == 'Y'")
paid_reviews_T_count = paid_reviews_T.count()
paid_reviews_T_count

285

In [13]:
unpaid_reviews_T = vote_count_50.filter ("vine == 'N'")
unpaid_reviews_T_count = unpaid_reviews_T.count()
unpaid_reviews_T_count

31542

In [14]:
Five_star_reviews = vote_count_50.filter("star_rating == 5")
Five_star_reviews_count = Five_star_reviews.count()
Five_star_reviews_count

14777

In [15]:
paid_5star_reviews = paid_reviews_T.filter("star_rating == 5")
paid_5star_reviews_count = paid_5star_reviews.count()
paid_5star_reviews_count

163

In [16]:
unpaid_5star_reviews = unpaid_reviews_T.filter("star_rating == 5")
unpaid_5star_reviews_count = unpaid_5star_reviews.count()
unpaid_5star_reviews_count

14614

In [17]:
paid_5star_percent = (paid_5star_reviews_count/paid_reviews_T_count) * 100 
round(paid_5star_percent, 2)

57.19

In [18]:
percent_total_5star = (paid_5star_reviews_count/Five_star_reviews_count) * 100
round(percent_total_5star, 2)

1.1

In [19]:
unpaid_vs_unpaid_total = (unpaid_5star_reviews_count/unpaid_reviews_T_count) * 100
round(unpaid_vs_unpaid_total, 3)

46.332

In [20]:
unpaid_vs_5star = (unpaid_5star_reviews_count/Five_star_reviews_count) * 100
round(unpaid_vs_5star, 3)

98.897